In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
import os

### Read the raw dataframe

In [3]:
raw_df = pd.read_csv('data/master_data/john_hopkins_research.csv')
raw_df.drop('Unnamed: 0', axis= 1, inplace= True)

## make certain Last Update is datetime
raw_df['Last Update'] = pd.to_datetime(raw_df['Last Update'])

### US dataframe

In [4]:
us_df = raw_df[raw_df['Country/Region'] == 'US']

## Clean the US df

In [5]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [6]:
def get_admin(combined_key):
    if pd.isnull(combined_key):
        county = np.nan
    else:
        county = combined_key.split(',')[0]
    return county

In [7]:
def clean_state(state):
    if 'D.C.' in state or 'U.S.' in state:
        state = state
    elif '(From Diamond Princess)' in state:
        if ',' in state:
            state = abbrev_us_state[state.split('(')[0].split(',')[-1][1:-1]]
        else:
            state = 'Diamond Princess'
    elif ',' in state:
        state = abbrev_us_state[state.split(',')[-1].replace(' ', '')]
        
    return state

In [8]:
us_df['Admin2'] = us_df['Combined_Key'].apply(get_admin)

C:\Users\sahil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
us_df['Province/State'] = us_df['Province/State'].apply(clean_state)

C:\Users\sahil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
### US at the state level
### TODO: group by after extracting data
us_df['Date'] = us_df['Last Update'].dt.date
us_states = us_df.groupby(['Province/State', 'Date']).sum()

C:\Users\sahil\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
np.unique(list(map(lambda x: x[0], us_states.index)))

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Chicago', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Grand Princess Cruise Ship', 'Guam', 'Hawaii',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Northern Mariana Islands',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico',
       'Recovered', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'US', 'United States Virgin Islands', 'Utah',
       'Vermont', 'Virgin Islands', 'Virgin Islands, U.S.', 'Virginia',
       'Washington', 'Washington, D.C.', 'West Virginia', 'Wisconsin',
      

In [12]:
us_states = us_states.reset_index()

In [13]:
us_states = us_states[['Province/State', 'Date', 'Confirmed', 'Deaths', 'Recovered', 'Active']]

### Initialize US transformed dataframe

In [14]:
us_transformed = us_states[['Province/State']].copy()

In [15]:
us_transformed.rename(columns={'Province/State': 'State'}, inplace=True)

In [16]:
us_transformed['ConfirmedToDate'] = us_states['Confirmed']

In [17]:
us_transformed['Active'] = us_states['Active']

In [18]:
us_transformed['DeathToDate'] = us_states['Deaths']

In [19]:
us_transformed['RecoveredToDate'] = us_states['Recovered']

In [20]:
us_transformed['Date'] = us_states['Date']

### Functions to help with transformations

In [21]:
copy_us_transformed = us_transformed.copy()
def new_type(col):
    copy_us_transformed['new'] = us_transformed.groupby('State').diff()[col]
    
    to_ret = np.where(np.isnan(copy_us_transformed['new']), \
                                         copy_us_transformed[col], \
                                         copy_us_transformed['new'])
    return to_ret

In [22]:
def prev_type(col):
    return us_transformed.groupby('State').shift()[col].fillna(0)

In [23]:
def get_rate(col1, col2):
    rate = []
    for state in us_transformed['State'].unique():
        tmp = us_transformed[us_transformed['State'] == state]
        rate.extend(tmp[col1]/tmp[col2].shift())
    return np.array(rate)

In [24]:
def get_days_in(col, num=1):
    return_array = np.array([])
    for state in us_transformed['State'].unique():
        temp_df = us_transformed[us_transformed['State'] == state].copy()
        arr = np.cumsum(np.where(temp_df[col] > num, 1, 0))
        return_array = np.append(return_array, arr)
    return return_array

### Transform Function

In [25]:
def transform(col, suffix):
    
    ### new cases
    new_col = 'New' + suffix
    us_transformed[new_col] = new_type(col)
    
    ### prev new cases
    prev_new_col = 'Prev' + new_col
    us_transformed[prev_new_col] = prev_type(new_col)
    
    ### cases growth
    growth_col = 'GrowthRate' + suffix
    us_transformed[growth_col] = get_rate(new_col, col)
    
    ### new cases two day avg
    col_name_2_day_avg = new_col + "2DayAvg"
    us_transformed[col_name_2_day_avg] = (us_transformed[new_col] + us_transformed[prev_new_col])/2
    
    ### 2 day cases growth
    col_name = 'GrowthRate' + suffix + "2DayAvg"
    us_transformed[col_name] = get_rate(col_name_2_day_avg, col)
    
    ### growth rate change
    col_growth_change = growth_col + 'Change'
    us_transformed[col_growth_change] = us_transformed.groupby('State').diff()[col_name]
    
    ### days to double
    days_to_double = 'DaysToDouble' + suffix
    us_transformed[days_to_double] = 0.72/us_transformed[growth_col]
    
    ### days to double 2 day average
    days_to_double_2 = days_to_double + '2DayAvg'
    us_transformed[days_to_double_2] = 0.72/us_transformed[col_name]
    
    ### days to double change
    days_to_double_2_change = days_to_double_2 + 'Change'
    us_transformed[days_to_double_2_change] = us_transformed.groupby('State').diff()[days_to_double_2]
    
    ### days in 1
    daysin1 = 'Daysin1' + suffix
    us_transformed[daysin1] = get_days_in(col, 1)
    
    ### days in 5
    daysin5 = 'Daysin5' + suffix
    us_transformed[daysin5] = get_days_in(col, 5)
    
    ### days in 100
    daysin100 = 'Daysin100' + suffix
    us_transformed[daysin100] = get_days_in(col, 100)
    
    
    ### days in 250
    daysin250 = 'Daysin250' + suffix
    us_transformed[daysin250] = get_days_in(col, 250)
    
    
    ### days in 1000
    daysin1000 = 'Daysin1000' + suffix
    us_transformed[daysin1000] = get_days_in(col, 1000)
    

### New Cases

In [26]:
us_transformed['NewConfirmed'] = us_transformed.groupby('State').diff()['ConfirmedToDate']

In [27]:
us_transformed['NewConfirmed'] = np.where(np.isnan(us_transformed['NewConfirmed']), \
                                         us_transformed['ConfirmedToDate'], \
                                         us_transformed['NewConfirmed'])

In [28]:
us_transformed['PrevNewConfirmed'] = us_transformed.groupby('State').shift()['NewConfirmed'].fillna(0)

In [29]:
us_transformed['GrowthRate'] = get_rate('NewConfirmed', 'ConfirmedToDate')

In [30]:
us_transformed['GrowthRate'] = us_transformed['GrowthRate'].fillna(1)

In [31]:
## new confirmed two day average
us_transformed['NewConfirmed2DayAvg'] = (us_transformed['NewConfirmed'] + us_transformed['PrevNewConfirmed'])/2

In [32]:
### growth rate two day average
us_transformed['GrowthRate2DayAvg'] = get_rate('NewConfirmed2DayAvg', 'ConfirmedToDate')

In [33]:
us_transformed['GrowthRate2DayAvg'] = us_transformed['GrowthRate2DayAvg'].fillna(1)

In [34]:
## growth rate change
us_transformed['GrowthRateChange'] = us_transformed.groupby('State').diff()['GrowthRate2DayAvg']

In [35]:
### Rule of 72
us_transformed['DaysToDoubleNew'] = 0.72/us_transformed['GrowthRate']

In [36]:
us_transformed['DaysToDouble2DayAvgNew'] = 0.72/us_transformed['GrowthRate2DayAvg']

In [37]:
us_transformed['DaysToDouble2DayAvgNewChange'] = us_transformed.groupby('State').diff()['DaysToDouble2DayAvgNew']

In [38]:
us_transformed['Daysin1'] = get_days_in('ConfirmedToDate', 1)
us_transformed['Daysin5'] = get_days_in('ConfirmedToDate', 5)
us_transformed['Daysin100'] = get_days_in('ConfirmedToDate', 100)
us_transformed['Daysin250'] = get_days_in('ConfirmedToDate', 250)
us_transformed['Daysin1000'] = get_days_in('ConfirmedToDate', 1000)

### Transform Death Data 

In [39]:
transform('DeathToDate', 'Deaths')

### Active Cases

In [40]:
us_transformed['Active'] = us_states['Active']

### Save the US data frame

In [41]:
if 'transformed' not in os.listdir('data'):
    os.mkdir('data/transformed')

In [42]:
us_transformed.to_csv('data/transformed/us_transformed.csv')

### Country DF

In [43]:
df_ = pd.read_csv('data/master_data/country_level/new_cases.csv')

In [44]:
df_2 = pd.read_csv('data/master_data/country_level/new_deaths.csv')

In [45]:
df_ = df_.melt(id_vars='date', value_name='NewConfirmed').rename(columns={'date':'Date'})

In [46]:
df_2 = df_2.melt(id_vars='date', value_name='NewDeaths').rename(columns={'date':'Date'})

In [47]:
df = df_.merge(df_2, on=['Date', 'variable']).rename(columns={'variable':'Country'})

In [48]:
df.groupby('Country').sum().to_csv('data/transformed/current.csv')

In [49]:
#### TODO: Check new confirmed negative 